# Finetuning with Trainer and text-classification model validation

In [1]:
# Current data source: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp/data
# Original notebook created by http://bkonat[dot]home[dot]amu[dot]edu[dot]pl

In [2]:
# !pip install transformers[torch]
# !pip install accelerate -U
#ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U

In [3]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import seaborn as sn
import torch

from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments, DataCollatorWithPadding, AutoTokenizer, pipeline

# Load data via datasets package and prepare _label to int_ dict 

In [4]:
root_dir = "data/kaggle_archive"
convert_to_csv = True

files = {'train': os.path.join(root_dir,'train.txt'),
         'test': os.path.join(root_dir,'test.txt'),
         'validate': os.path.join(root_dir,'val.txt')
         }

## Get target map 
#Section assumes that file contains all unique labels
ds_dict = {}
for k,v in files.items():
    ds_dict[k] = pd.read_csv(v, sep=';', header=None, names=['sentence', 'label'])
    
target_map = {k:i for i,k in enumerate(ds_dict['train']['label'].unique())}
print(target_map)

for k,v in ds_dict.items():
    v['label'] = v['label'].map(target_map)
    
ds = DatasetDict({k: Dataset.from_pandas(v) for k,v in ds_dict.items()})
ds

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2000
    })
    validate: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2000
    })
})

## Finetuning

### Configuring model and tokenizer

In [5]:
# model name or path to model
# model_path = 'SamLowe/roberta-base-go_emotions'
model_path = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

tokenized_datasets = ds.map(tokenize_fn, batched=True)
tokenized_datasets

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    validate: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [6]:
config = AutoConfig.from_pretrained(model_path)

config.id2label = {v:k for k, v in target_map.items()}
config.label2id = target_map

config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "sadness",
    "1": "anger",
    "2": "love",
    "3": "surprise",
    "4": "fear",
    "5": "joy"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anger": 1,
    "fear": 4,
    "joy": 5,
    "love": 2,
    "sadness": 0,
    "surprise": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.38.2",
  "vocab_size": 30522
}

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, config=config, ignore_mismatched_sizes=True)

#If we get info Model's device: cpu - need to move model to gpu
#Check if your model runs on gpu (this should happen automatically, when gpu is available)
device = next(model.parameters()).device
print("Model's device:", device)

if torch.cuda.is_available():
    model = model.cuda()
    print("Running on GPU")
else:
    print("CUDA (GPU support) is not available. Model will run on CPU.")

summary(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model's device: cpu
Running on GPU


Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           4,614
├─Dropout: 1-4                                          --
Total params: 66,958,086
Trainable params: 66,958,086
Non-trainable params: 0

### Training

In [8]:
trained_model_path = f'output/models/{model_path}_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")}'

training_args = TrainingArguments(
  output_dir=f'{trained_model_path}/checkpoints',
  evaluation_strategy='epoch',
  logging_strategy='epoch',
  save_strategy='epoch',
  num_train_epochs=10,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
)

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average='macro')
  return {'accuracy': acc, 'f1': f1}

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.380500,0.223676,0.918000,0.879410
2,0.143700,0.180085,0.929500,0.885094
3,0.115400,0.194072,0.926000,0.883133
4,0.090200,0.262936,0.924000,0.881260
5,0.060800,0.266639,0.931000,0.895203
6,0.036000,0.366909,0.925500,0.876207
7,0.024000,0.375555,0.927000,0.883184
8,0.014700,0.433104,0.927000,0.884394
9,0.010900,0.436206,0.928500,0.887293
10,0.006200,0.472601,0.926000,0.882930


TrainOutput(global_step=10000, training_loss=0.08823961234092713, metrics={'train_runtime': 342.9861, 'train_samples_per_second': 466.491, 'train_steps_per_second': 29.156, 'total_flos': 1945612950228864.0, 'train_loss': 0.08823961234092713, 'epoch': 10.0})

### Classification

In [9]:
# Get last epoch number
checkpoint_no = [int(i.rsplit('-')[-1]) for i in os.listdir(f'{trained_model_path}/checkpoints')]
checkpoint_no.sort(reverse=True)
checkpoint_no

[10000, 9000, 8000, 7000, 6000, 5000, 4000, 3000, 2000, 1000]

In [10]:
#Inspect above results to decide which checkpoint to use, by default last is taken
chosen_checkpoint = checkpoint_no[0]
checkpoint_path = f'{trained_model_path}/checkpoints/checkpoint-{chosen_checkpoint}'

task = 'text-classification'

classifier = pipeline(task,
                      model=checkpoint_path,
                      device=0)

classifier.save_pretrained(f'{trained_model_path}/pipeline')


In [11]:
pred_val = classifier(ds['validate']['sentence'])

ds_dict['validate']['predicted'] = [i['label'] for i in pred_val]
ds_dict['validate']['label'] = ds_dict['validate']['label'].map({v:k for k, v in target_map.items()})

ds_dict['validate'].to_csv(f'{trained_model_path}/validated.csv', index=False)

ds_dict['validate']

,sentence,label,predicted
0,im feeling quite sad and sorry for myself but ...,sadness,sadness
1,i feel like i am still looking at a blank canv...,sadness,sadness
2,i feel like a faithful servant,love,joy
3,i am just feeling cranky and blue,anger,anger
4,i can have for a treat or if i am feeling festive,joy,joy
...,...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness,sadness
1996,i constantly worry about their fight against n...,joy,joy
1997,i feel its important to share this info for th...,joy,joy
1998,i truly feel that if you are passionate enough...,joy,joy


In [12]:
print("acc:", accuracy_score(ds['validate']['label'], pred_val))
print("f1:", f1_score(ds['validate']['label'], pred_val, average='macro'))

def plot_cm(cm):
    classes = list(target_map.keys())
    df_cm = pd.DataFrame(cm, index=classes, columns=classes)
    ax = sn.heatmap(df_cm, annot=True, fmt='.2g')
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Target")

cm = confusion_matrix(ds['validate']['label'], pred_val, normalize='true')
plot_cm(cm)

ValueError: Classification metrics can't handle a mix of multiclass and unknown targets